# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from logtail import LogtailHandler
import logging
import os

In [ ]:

#| export
def setup_logging():
    """
    Configure logging to use Better Stack's LogtailHandler
    with env context included automatically.
    
    This should be called once at application startup.
    """
    env = os.environ.get("ENV", "development")
    source_token = os.environ.get("BETTER_STACK_SOURCE_TOKEN")
    host = os.environ.get("BETTER_STACK_HOST")
    
    if not source_token or not host:
        # If Better Stack credentials are not available, fall back to basic config
        logging.basicConfig(level=logging.INFO)
        logging.warning("Better Stack credentials not found, using basic logging config")
        return
    
    # Configure the root logger first to set the minimum level
    root_logger = logging.getLogger()
    root_logger.setLevel(logging.INFO)
    
    # Remove any existing handlers to avoid duplicates
    for handler in root_logger.handlers[:]:
        root_logger.removeHandler(handler)
    
    # Add a console handler for local development
    console = logging.StreamHandler()
    root_logger.addHandler(console)
    
    try:
        # Set up the Logtail handler
        handler = LogtailHandler(
            source_token=source_token,
            host=host
        )
        
        # Override LogRecord factory to add env to all records
        old_factory = logging.getLogRecordFactory()
        def record_factory(*args, **kwargs):
            record = old_factory(*args, **kwargs)
            record.env = env
            return record
        logging.setLogRecordFactory(record_factory)
        
        # Add the handler to the root logger
        root_logger.addHandler(handler)
        
        logging.info(f"Better Stack logging configured with env: {env}")
    except Exception as e:
        logging.error(f"Failed to configure Better Stack logging: {e}")



In [ ]:
#| export 
# If this file is run directly, test the logging
if __name__ == '__main__':
    setup_logging()
    logger = logging.getLogger(__name__)
    logger.info("Test log message")
    logger.error("Test error message") 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()